<a href="https://colab.research.google.com/github/everaldo/algorithmic_trading_practice_project/blob/master/algorithmic_trading_report.ipynb"
target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Algorithmic Trading Project Report
**Click the badge above to open in Google Colab**

# Algorithmic Trading Project Report

**Author:** Everaldo Gomes  
**Date:** August 9, 2025  
**Project:**: ALGORITHMIC TRADING PRACTICE PROJECT 01 - Brazilian Cryptocurrency Algorithmic Trading System  

## Current Understanding of Central Topics and Tools

Through this project, I developed comprehensive understanding of:

**Machine Learning for Finance:** Implemented BaggingRegressor ensemble combining DecisionTree (60%) + MLP (40%) with 73 features including SMA analysis, log returns, momentum, volatility, and 6-period lags. Results showed -3.21% return vs +9.56% buy & hold, highlighting the complexity of financial prediction.

**Python Scientific Stack:** Gained hands-on proficiency with numpy, pandas, scikit-learn through practical application. Key learning included NaN handling with forward/backward fill, feature engineering pipelines, and model ensemble techniques.

**API Integration:** Developed complete mercado_bitcoin_python library (v0.2.7) with authorization token authentication, extending from basic endpoints to full trading capabilities. Achieved 79 tests coverage through systematic debugging.

**Production Systems:** Implemented comprehensive logging architecture, timeframe interpolation (5m candles), anti-loss protection considering trading fees (0.75% minimum profit), and Telegram notifications.

```python
# Key ML implementation insight
def train_once(self, historical_data: pd.DataFrame):
    # Single training without retraining (book approach)
    X, y = self.prepare_ml_data(historical_data)
    split = int(len(X) * 0.7)  # 70/30 sequential split
    X_train, X_test = X.iloc[:split], X.iloc[split:]
    
    # Handle NaN values - critical for financial data
    nan_mask = pd.isna(X_train).any(axis=1) | pd.isna(y_train)
    X_train, y_train = X_train[~nan_mask], y_train[~nan_mask]
```

## Strengths and Weaknesses Identified

**Strengths:**
- **System Engineering:** Successfully built production-ready infrastructure with proper error handling, logging, and testing
- **Problem-Solving:** Overcame Brazilian regulatory restrictions by developing custom API library: since OANDA is not available in Brazil, I used Mercado Bitcoin for the experiments
- **AI Collaboration:** Effectively leveraged Claude Code for rapid prototyping while maintaining code quality

**Weaknesses:**
- **Feature Engineering:** Mixed different paradigms (AdaBoost book features + traditional technical indicators) which may have hindered ML performance
- **Market Intuition:** Limited experience with cryptocurrency market dynamics led to oversimplified prediction targets
- **Model Validation:** Insufficient backtesting period and inadequate warmup (55 candles) for complex feature stabilization

## Experiences Working Independently 

**Technical Background:** Computer Science (UFPR 2007), Master's (2010), 2 months experience with the Backtrader library (Nov/Dec 2024), but limited numpy/pandas hands-on experience before this project.

**Independent Development:** Successfully managed complete project lifecycle from conception to deployment. Live trading session (6 hours, SOL-BRL) generated 100% sell signals during Friday evening weak market conditions, demonstrating system's conservative behavior.

**Learning Approach:** Combined book methodology ("Python for Algorithmic Trading" pages 298-301) with practical system requirements, though this hybrid approach may have compromised ML effectiveness.

## Suggestions for Improving Existing Materials and Resources

**Enhanced Learning Path:**
1. **Timeframe Considerations:** Materials should emphasize the critical importance of timeframe selection and warmup periods for feature stability
2. **Feature Engineering Guidance:** Clearer distinction between different ML paradigms (AdaBoost vs ensemble methods) and when to combine them
3. **Production Readiness:** More emphasis on error handling, logging, and testing in financial applications

**Practical Improvements:**
- Include Brazilian market examples and regulatory considerations
- Provide template for API wrapper development when standard brokers unavailable  
- Add section on timeframe interpolation techniques for missing candle periods

## Additional Comments and Observations

**Key Insight:** Simple technical strategies often outperform complex ML models in volatile crypto markets. The project's main value was in building robust infrastructure rather than profitable predictions.

**Claude Code Experience:** AI-assisted development proved exceptionally effective for:
- Automatic best practices implementation
- Real-time learning of scientific computing patterns  
- Systematic debugging of complex integration issues

**Market Reality:** Friday evening trading conditions with SOL at 20-day momentum low validated system's conservative approach through predominant sell signals.

**Future Focus:** Asset momentum screening across multiple pairs would be more valuable than fixed-asset complex feature engineering.

---

*This independent work demonstrates modern collaborative AI development methodologies while highlighting both possibilities and limitations of ML in cryptocurrency trading.*


"""
Intelligent Trading System with Anti-Loss Protection
==================================================

Features:
- Smart account info collection (accountId, tier, fees)
- Anti-loss protection (don't sell at loss considering fees)
- Telegram integration for real-time notifications
- Budget management (R$100 split into max 5 positions)
- Comprehensive logging at start, during trades, and end

Author: Everaldo Gomes
Date: 2025-08-08
"""

In [ ]:
 # 📦 Dependencies Installation for Google Colab
# Run this cell first to install all required packages

# Core dependencies
%pip install pandas>=2.3.0 numpy>=2.3.0 scikit-learn>=1.7.0
%pip install aiohttp>=3.12.0 structlog>=25.4.0 python-dotenv>=1.1.0
%pip install nest-asyncio>=1.6.0

# Install mercado-bitcoin-python from repository
GITHUB_REPO = "https://raw.githubusercontent.com/everaldo/algorithmic_trading_practice_project/master"
WHEEL_FILE = "mercado_bitcoin_python-0.2.8-py3-none-any.whl"
%pip install "{GITHUB_REPO}/{WHEEL_FILE}"

# Configure asyncio for notebooks
import nest_asyncio
nest_asyncio.apply()

print("✅ All dependencies installed successfully!")

In [ ]:
# 🔐 Complete Environment Setup for Google Colab
# Copy this entire cell to the top of your Jupyter notebook

import os
from pathlib import Path

def setup_complete_environment():
    """
    Setup ALL environment variables for the intelligent trading system
    Handles all 19 variables from .env.example with fallback to demo mode
    """
    
    # Check if we're in Colab
    try:
        from google.colab import userdata
        print("🔐 Google Colab detected - using Colab Secrets...")
        
        try:
            # System Mode
            os.environ['MODE'] = userdata.get('MODE', 'test')
            os.environ['DEBUG'] = userdata.get('DEBUG', 'true')
            
            # Trading Configuration
            os.environ['TOTAL_BUDGET'] = userdata.get('TOTAL_BUDGET', '100.0')
            os.environ['MAX_POSITIONS'] = userdata.get('MAX_POSITIONS', '5')
            os.environ['POSITION_SIZE'] = userdata.get('POSITION_SIZE', '20.0')
            os.environ['ASSET_SYMBOL'] = userdata.get('ASSET_SYMBOL', 'SOL-BRL')
            
            # Mercado Bitcoin API
            os.environ['MB_API_KEY'] = userdata.get('MB_API_KEY')
            os.environ['MB_API_SECRET'] = userdata.get('MB_API_SECRET')
            os.environ['MB_READ_ONLY'] = userdata.get('MB_READ_ONLY', 'true')
            
            # Database
            os.environ['DATABASE_PATH'] = userdata.get('DATABASE_PATH', 'data/trading.db')
            
            # Logging
            os.environ['LOG_LEVEL'] = userdata.get('LOG_LEVEL', 'INFO')
            os.environ['LOG_PATH'] = userdata.get('LOG_PATH', 'logs/')
            
            # Telegram Configuration
            os.environ['TELEGRAM_ENABLED'] = userdata.get('TELEGRAM_ENABLED', 'false')
            os.environ['TELEGRAM_BOT_TOKEN'] = userdata.get('TELEGRAM_BOT_TOKEN', 'demo_token')
            os.environ['TELEGRAM_CHAT_ID'] = userdata.get('TELEGRAM_CHAT_ID', 'demo_chat')
            os.environ['TELEGRAM_AUTHORIZED_USERS'] = userdata.get('TELEGRAM_AUTHORIZED_USERS', 'demo_user')
            
            # ML Strategy Settings
            os.environ['MODEL_PATH'] = userdata.get('MODEL_PATH', 'models/')
            os.environ['DATA_PATH'] = userdata.get('DATA_PATH', 'data/')
            os.environ['TRAINING_TEST_SPLIT'] = userdata.get('TRAINING_TEST_SPLIT', '0.3')
            
            # Session Settings
            os.environ['SESSION_DURATION_HOURS'] = userdata.get('SESSION_DURATION_HOURS', '6')
            os.environ['DATA_COLLECTION_BARS'] = userdata.get('DATA_COLLECTION_BARS', '2016')
            
            print("✅ Successfully loaded from Colab secrets")
            return True
            
        except Exception as e:
            print(f"⚠️  Some Colab secrets missing: {e}")
            
    except ImportError:
        print("📓 Not in Colab - checking for .env...")
        try:
            from dotenv import load_dotenv
            load_dotenv('.env')
            print("✅ Loaded .env file")
            return True
        except:
            pass
    
    # DEMO MODE - All variables with safe defaults
    print("🧪 Setting up DEMO MODE...")
    
    # System Mode
    os.environ['MODE'] = 'demo'
    os.environ['DEBUG'] = 'true'
    
    # Trading Configuration
    os.environ['TOTAL_BUDGET'] = '100.0'
    os.environ['MAX_POSITIONS'] = '5'
    os.environ['POSITION_SIZE'] = '20.0'
    os.environ['ASSET_SYMBOL'] = 'SOL-BRL'
    
    # Mercado Bitcoin API (DEMO)
    os.environ['MB_API_KEY'] = 'demo_api_key_safe_for_sharing'
    os.environ['MB_API_SECRET'] = 'demo_secret_safe_for_sharing'
    os.environ['MB_READ_ONLY'] = 'true'
    
    # Database (Colab-friendly paths)
    os.environ['DATABASE_PATH'] = '/content/data/trading.db'
    
    # Logging
    os.environ['LOG_LEVEL'] = 'INFO'
    os.environ['LOG_PATH'] = '/content/logs/'
    
    # Telegram (disabled in demo)
    os.environ['TELEGRAM_ENABLED'] = 'false'
    os.environ['TELEGRAM_BOT_TOKEN'] = 'demo_bot_token_not_real'
    os.environ['TELEGRAM_CHAT_ID'] = 'demo_chat_id'
    os.environ['TELEGRAM_AUTHORIZED_USERS'] = 'demo_users'
    
    # ML Strategy Settings
    os.environ['MODEL_PATH'] = '/content/models/'
    os.environ['DATA_PATH'] = '/content/data/'
    os.environ['TRAINING_TEST_SPLIT'] = '0.3'
    
    # Session Settings
    os.environ['SESSION_DURATION_HOURS'] = '6'
    os.environ['DATA_COLLECTION_BARS'] = '2016'
    
    print("✅ DEMO environment ready - all API calls will be mocked")
    return False


def create_directories():
    """Create necessary directories for Colab"""
    dirs = [
        Path(os.environ.get('DATA_PATH', '/content/data')),
        Path(os.environ.get('LOG_PATH', '/content/logs')),
        Path(os.environ.get('MODEL_PATH', '/content/models'))
    ]
    
    for directory in dirs:
        directory.mkdir(exist_ok=True, parents=True)
        print(f"📁 Created: {directory}")


def print_config_summary():
    """Print configuration summary"""
    print("\n" + "="*50)
    print("📊 CONFIGURATION SUMMARY")
    print("="*50)
    
    # Key settings
    key_vars = [
        ('MODE', 'System Mode'),
        ('ASSET_SYMBOL', 'Trading Asset'), 
        ('TOTAL_BUDGET', 'Budget'),
        ('MB_READ_ONLY', 'Safety Mode'),
        ('TELEGRAM_ENABLED', 'Notifications'),
        ('LOG_LEVEL', 'Logging'),
        ('SESSION_DURATION_HOURS', 'Session Length')
    ]
    
    for var, desc in key_vars:
        value = os.environ.get(var, 'NOT SET')
        print(f"  {desc}: {value}")
    
    print("="*50)
    
    # Safety notice
    mode = os.environ.get('MODE', 'unknown')
    read_only = os.environ.get('MB_READ_ONLY', 'unknown')
    
    if mode == 'demo':
        print("🧪 DEMO MODE: Safe for sharing, no real trades")
    elif read_only == 'true':
        print("🔒 READ-ONLY: Safe mode, no actual trades")
    else:
        print("⚠️  LIVE MODE: Real trading enabled!")


# MAIN EXECUTION - Run this cell first!
print("🚀 INTELLIGENT TRADING SYSTEM - Environment Setup")
print("="*60)

# Setup environment
real_mode = setup_complete_environment()

# Create directories
create_directories()

# Print summary
print_config_summary()

# Configure asyncio for notebooks
import nest_asyncio
nest_asyncio.apply()

print(f"\n✅ Environment ready for {'REAL' if real_mode else 'DEMO'} trading!")
print("🔄 Asyncio configured for notebook compatibility")
print("="*60)

In [ ]:
import asyncio
import sys
import sqlite3
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional, Any
from dataclasses import dataclass
import structlog
import pandas as pd

# Import from local wheel
from mercado_bitcoin_python import MBTpqoa
from mercado_bitcoin_python.api.trading_endpoints import AccountInfo, TradingFees, Balance

In [ ]:
"""
Simple Configuration for Intelligent Trading System
"""

import os
from dataclasses import dataclass
from typing import Optional
import structlog
from pathlib import Path

logger = structlog.get_logger()


@dataclass
class TradingConfig:
    """Simple trading configuration from environment variables."""
    # System
    mode: str = "test"
    debug: bool = True
    
    # Trading
    total_budget: float = 100.0
    max_positions: int = 5
    position_size: float = 20.0
    asset_symbol: str = "SOL-BRL"
    
    # API
    mb_api_key: Optional[str] = None
    mb_api_secret: Optional[str] = None
    mb_read_only: bool = True
    
    # Telegram
    telegram_enabled: bool = False
    telegram_bot_token: Optional[str] = None
    telegram_chat_id: Optional[str] = None
    telegram_authorized_users: str = ""
    
    # Paths
    database_path: str = "data/trading.db"
    model_path: str = "models/"
    data_path: str = "data/"
    log_path: str = "logs/"
    
    # ML Settings
    training_test_split: float = 0.3
    
    # Session
    session_duration_hours: int = 6
    data_collection_bars: int = 2016
    
    @classmethod
    def from_env(cls, env_file: Optional[str] = None) -> "TradingConfig":
        """Load configuration from environment variables."""
        # Load .env file if provided
        if env_file and Path(env_file).exists():
            from dotenv import load_dotenv
            load_dotenv(env_file)
        
        return cls(
            mode=os.getenv("MODE", "test"),
            debug=os.getenv("DEBUG", "true").lower() == "true",
            
            total_budget=float(os.getenv("TOTAL_BUDGET", "100.0")),
            max_positions=int(os.getenv("MAX_POSITIONS", "5")),
            position_size=float(os.getenv("POSITION_SIZE", "20.0")),
            asset_symbol=os.getenv("ASSET_SYMBOL", "SOL-BRL"),
            
            mb_api_key=os.getenv("MB_API_KEY"),
            mb_api_secret=os.getenv("MB_API_SECRET"),
            mb_read_only=os.getenv("MB_READ_ONLY", "true").lower() == "true",
            
            telegram_enabled=os.getenv("TELEGRAM_ENABLED", "false").lower() == "true",
            telegram_bot_token=os.getenv("TELEGRAM_BOT_TOKEN"),
            telegram_chat_id=os.getenv("TELEGRAM_CHAT_ID"),
            telegram_authorized_users=os.getenv("TELEGRAM_AUTHORIZED_USERS", ""),
            
            database_path=os.getenv("DATABASE_PATH", "data/trading.db"),
            model_path=os.getenv("MODEL_PATH", "models/"),
            data_path=os.getenv("DATA_PATH", "data/"),
            log_path=os.getenv("LOG_PATH", "logs/"),
            
            training_test_split=float(os.getenv("TRAINING_TEST_SPLIT", "0.3")),
            
            session_duration_hours=int(os.getenv("SESSION_DURATION_HOURS", "6")),
            data_collection_bars=int(os.getenv("DATA_COLLECTION_BARS", "2016"))
        )
    
    def create_directories(self):
        """Create necessary directories."""
        for path_attr in ['data_path', 'model_path', 'log_path']:
            path = Path(getattr(self, path_attr))
            path.mkdir(parents=True, exist_ok=True)
        
        # Ensure database directory exists
        Path(self.database_path).parent.mkdir(parents=True, exist_ok=True)
    
    def __post_init__(self):
        """Validate configuration after initialization."""
        if self.total_budget <= 0:
            raise ValueError("Total budget must be positive")
        
        if self.max_positions <= 0:
            raise ValueError("Max positions must be positive")
        
        if self.position_size <= 0:
            raise ValueError("Position size must be positive")
        
        if self.position_size * self.max_positions > self.total_budget:
            raise ValueError("Position size * max positions exceeds total budget")
        
        logger.info("Configuration loaded",
                   mode=self.mode,
                   budget=self.total_budget,
                   positions=self.max_positions,
                   telegram_enabled=self.telegram_enabled)

In [ ]:
"""
Simple Telegram Service for Intelligent Trading System
"""

import asyncio
import aiohttp
import structlog
from datetime import datetime
from typing import Optional, Dict, Any

logger = structlog.get_logger()


class TelegramService:
    """Simple Telegram service for notifications."""
    
    def __init__(self, config: TradingConfig):
        self.config = config
        self.enabled = config.telegram_enabled and config.telegram_bot_token
        self.bot_token = config.telegram_bot_token
        self.chat_id = config.telegram_chat_id
        self.authorized_users = [u.strip() for u in config.telegram_authorized_users.split(',') if u.strip()]
        
        self.logger = logger.bind(service="telegram")
        
        if self.enabled:
            self.logger.info("Telegram service enabled", 
                           has_chat_id=bool(self.chat_id),
                           authorized_users_count=len(self.authorized_users))
        else:
            self.logger.info("Telegram service disabled")
    
    async def send_message(self, text: str, chat_id: Optional[str] = None, 
                          parse_mode: str = "HTML") -> bool:
        """Send message to Telegram."""
        if not self.enabled:
            self.logger.debug("Telegram disabled - message not sent")
            return False
        
        target_chat = chat_id or self.chat_id
        if not target_chat:
            self.logger.error("No chat ID configured")
            return False
        
        url = f"https://api.telegram.org/bot{self.bot_token}/sendMessage"
        payload = {
            "chat_id": target_chat,
            "text": text,
            "parse_mode": parse_mode
        }
        
        try:
            async with aiohttp.ClientSession() as session:
                async with session.post(url, json=payload) as response:
                    if response.status == 200:
                        result = await response.json()
                        if result.get("ok"):
                            self.logger.debug("Message sent successfully")
                            return True
                        else:
                            self.logger.error("Telegram API error", 
                                            description=result.get("description"))
                            return False
                    else:
                        self.logger.error("HTTP error sending message", 
                                        status=response.status)
                        return False
        
        except Exception as e:
            self.logger.error("Exception sending message", error=str(e))
            return False
    
    async def send_session_start(self, account_summary: Dict[str, Any], session_info: Dict[str, Any]):
        """Send trading session start notification."""
        if not self.enabled:
            return False
        
        message = f"""🚀 <b>INTELLIGENT TRADING SESSION STARTED</b>

📊 <b>Account Information:</b>
• Account ID: {account_summary.get('account_id', 'N/A')[:8]}...
• Tier: {account_summary.get('tier', 'N/A')}
• Symbol: {account_summary.get('symbol', 'SOL-BRL')}

💰 <b>Trading Fees (SOL-BRL):</b>
• Maker: {account_summary.get('fees', {}).get('maker', 0):.4f}%
• Taker: {account_summary.get('fees', {}).get('taker', 0):.4f}%
• Total Cycle: {account_summary.get('fees', {}).get('total_cycle', 0):.4f}%

💵 <b>Initial Balances:</b>
• SOL: {account_summary.get('balances', {}).get('SOL', 0):.6f}
• BRL: R$ {account_summary.get('balances', {}).get('BRL', 0):.2f}

🎯 <b>Session Config:</b>
• Budget: R$ {self.config.total_budget:.2f}
• Max positions: {self.config.max_positions} × R$ {self.config.position_size:.2f}
• Strategy: ML Ensemble (60% Tree + 40% MLP)
• Anti-loss protection: ON

⏰ <b>Started:</b> {session_info.get('start_time', datetime.now()).strftime('%H:%M:%S')}

#TradingSession #SOL #MLStrategy"""
        
        return await self.send_message(message)
    
    async def send_signal_alert(self, signal: Dict[str, Any], position_info: Dict[str, Any]):
        """Send trading signal notification."""
        if not self.enabled:
            return False
        
        signal_type = signal['signal_type'].upper()
        emoji = "🟢" if signal_type == "BUY" else "🔴"
        
        message = f"""{emoji} <b>{signal_type} SIGNAL</b>

💰 <b>SOL-BRL:</b> R$ {signal['price']:.2f}
📊 <b>Confidence:</b> {signal['confidence']:.1%}
🤖 <b>ML Predictions:</b>
• Tree: {signal.get('predictions', {}).get('tree', 0):.3f}
• MLP: {signal.get('predictions', {}).get('mlp', 0):.3f}
• Final: {signal.get('predictions', {}).get('final', 0):.3f}

💵 <b>Position:</b> R$ {position_info['position_size']:.2f}
📈 <b>Quantity:</b> {position_info['quantity']:.6f} SOL

⏰ {datetime.now().strftime('%H:%M:%S')}

#Signal #{signal_type} #SOL"""
        
        return await self.send_message(message)
    
    async def send_trade_execution(self, trade_result: Dict[str, Any], is_profitable: Optional[bool] = None):
        """Send trade execution notification."""
        if not self.enabled:
            return False
        
        side = trade_result['side'].upper()
        emoji = "✅" if trade_result['status'] == 'filled' else "⚠️"
        
        profit_info = ""
        if is_profitable is not None:
            profit_emoji = "📈" if is_profitable else "📉"
            profit_info = f"\n{profit_emoji} <b>Profitable:</b> {'Yes' if is_profitable else 'No (Protected)'}"
        
        message = f"""{emoji} <b>TRADE EXECUTED</b>

📊 <b>Order:</b> {side} SOL-BRL
💰 <b>Price:</b> R$ {trade_result['price']:.2f}
📈 <b>Quantity:</b> {trade_result['quantity']:.6f} SOL
💵 <b>Value:</b> R$ {trade_result['value']:.2f}
💸 <b>Fees:</b> R$ {trade_result['fees']:.2f}{profit_info}

🔖 <b>Order ID:</b> {trade_result['order_id'][:12]}...
⏰ {datetime.now().strftime('%H:%M:%S')}

#Trade #{side} #Executed"""
        
        return await self.send_message(message)
    
    async def send_protection_alert(self, position: Dict[str, Any], current_price: float, reason: str):
        """Send anti-loss protection alert."""
        if not self.enabled:
            return False
        
        message = f"""🛡️ <b>ANTI-LOSS PROTECTION</b>

Sell signal rejected to prevent loss:
• Entry: R$ {position['entry_price']:.2f}
• Current: R$ {current_price:.2f}
• Breakeven: R$ {position.get('breakeven_price', 0):.2f}

📝 <b>Reason:</b> {reason}

⏰ {datetime.now().strftime('%H:%M:%S')}

#Protection #NoLoss #SOL"""
        
        return await self.send_message(message)
    
    async def send_session_end(self, session_summary: Dict[str, Any], final_balances: Dict[str, float]):
        """Send trading session end notification."""
        if not self.enabled:
            return False
        
        profit_emoji = "📈" if session_summary.get('net_pnl', 0) >= 0 else "📉"
        
        message = f"""🏁 <b>TRADING SESSION ENDED</b>

📊 <b>Session Summary:</b>
• Duration: {session_summary.get('duration_hours', 0):.1f}h
• Total trades: {session_summary.get('total_trades', 0)}
• Win rate: {session_summary.get('win_rate', 0):.1f}%

{profit_emoji} <b>Performance:</b>
• Initial: R$ {session_summary.get('initial_balance', 0):.2f}
• Final: R$ {session_summary.get('final_balance', 0):.2f}
• P&L: R$ {session_summary.get('net_pnl', 0):+.2f} ({session_summary.get('roi_pct', 0):+.2f}%)
• Fees paid: R$ {session_summary.get('total_fees_paid', 0):.2f}

💰 <b>Final Balances:</b>
• SOL: {final_balances.get('SOL', 0):.6f}
• BRL: R$ {final_balances.get('BRL', 0):.2f}

⏰ <b>Ended:</b> {session_summary.get('end_time', datetime.now()).strftime('%H:%M:%S')}

#TradingSession #Complete #Summary"""
        
        return await self.send_message(message)

In [ ]:
"""
Simplified Algorithmic Trading Exercise
=====================================

Implementation based on "Python for Algorithmic Trading" book:
- AdaBoost features (pages 298-301)
- Single training without retraining
- SOL-BRL with R$100 budget
- 6h session = 360 bars

Author: Everaldo Gomes
Date: 2025-08-08
"""

import numpy as np
import pandas as pd
import pickle
import structlog
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any
from dataclasses import dataclass
import asyncio

# Scikit-learn imports
from sklearn.ensemble import BaggingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Imports locais
from mercado_bitcoin_python import MBTpqoa

# Configurar logging estruturado
logging = structlog.get_logger()


@dataclass
class MLTradingConfig:
    """Trading exercise configuration"""
    # Asset
    asset_symbol: str = "SOL-BRL"
    
    # Data
    historical_bars: int = 2016  # 1 week 5min = 5*12*24*7
    session_bars: int = 360      # 6 hours 5min = 5*12*6
    timeframe: str = "5m"
    
    # Budget
    total_budget: float = 100.0  # R$100
    max_positions: int = 5       # max 5 positions
    position_size: float = 20.0  # R$20 each
    
    # ML Model
    tree_weight: float = 0.6     # 60% DecisionTree
    mlp_weight: float = 0.4      # 40% MLP
    confidence_threshold: float = 0.55
    
    # Features (based on book)
    window_size: int = 20        # Rolling window
    lags: int = 6               # Temporal lags
    
    # Paths
    data_path: str = "data/exercise"
    model_path: str = "data/exercise/model.pkl"


class SimpleMLEnsembleStrategy:
    """
    Simplified ML Ensemble Strategy based on AdaBoost book.
    
    Inline features from book (pages 298-301):
    - SMA(10,20,50)
    - Log returns
    - Momentum & Volatility  
    - Price Min/Max
    - Volume Ratio
    - 6 period lags
    
    No retraining during execution (as in the book).
    """
    
    def __init__(self, config: MLTradingConfig):
        self.config = config
        self.logger = logging.bind(strategy="SimpleMLEnsemble", asset=config.asset_symbol)
        
        # Models
        self.tree_model: Optional[BaggingRegressor] = None
        self.mlp_model: Optional[BaggingRegressor] = None
        self.scaler = StandardScaler()
        self.is_trained = False
        
        # Features
        self.feature_columns: List[str] = []
        self.model_metrics: Dict[str, Any] = {}
        
        # Data
        self.data_path = Path(config.data_path)
        self.data_path.mkdir(parents=True, exist_ok=True)
        
        self.logger.info("SimpleMLEnsembleStrategy initialized", 
                        historical_bars=config.historical_bars,
                        session_bars=config.session_bars)
    
    def create_features_inline(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Create sophisticated features based on SMA strategies from live_trading_system.
        
        AdaBoost book features + advanced strategy logic:
        - SMA slopes (trend direction)
        - Cross detection & strength
        - Price position relative to SMAs
        - Distance strength
        - Volume analysis
        + 6-period lags
        """
        df = data.copy()
        window = self.config.window_size
        
        # 1. Log returns (core of AdaBoost in book)
        df['return'] = np.log(df['close'] / df['close'].shift(1))
        
        # 2. SMAs (chosen periods: 10, 20, 50)
        df['sma_10'] = df['close'].rolling(10).mean()
        df['sma_20'] = df['close'].rolling(20).mean()
        df['sma_50'] = df['close'].rolling(50).mean()
        
        # 3. AdaBoost book features
        df['momentum'] = df['return'].rolling(window).mean()
        df['volatility'] = df['return'].rolling(window).std()
        df['price_min'] = df['close'].rolling(window).min()
        df['price_max'] = df['close'].rolling(window).max()
        df['price_norm'] = (df['close'] - df['price_min']) / (df['price_max'] - df['price_min'])
        
        # 4. Volume ratio
        if 'volume' in df.columns:
            df['volume_avg'] = df['volume'].rolling(window).mean()
            df['volume_ratio'] = df['volume'] / df['volume_avg']
        else:
            df['volume_ratio'] = 1.0
        
        # ===== ADVANCED SMA STRATEGY FEATURES =====
        
        # 5. SMA Slopes (trend direction) - from live_trading_system
        df['sma_10_slope'] = self._calculate_slope(df['sma_10'])
        df['sma_20_slope'] = self._calculate_slope(df['sma_20'])
        df['sma_50_slope'] = self._calculate_slope(df['sma_50'])
        
        # 6. Cross Detection & Strength - from SMACrossoverStrategy
        df['cross_10_20'] = self._detect_cross(df['sma_10'], df['sma_20'])
        df['cross_20_50'] = self._detect_cross(df['sma_20'], df['sma_50'])
        df['cross_strength_10_20'] = abs((df['sma_10'] - df['sma_20']) / df['sma_20'])
        df['cross_strength_20_50'] = abs((df['sma_20'] - df['sma_50']) / df['sma_50'])
        
        # 7. Price Position - from SMACrossoverStrategy
        df['price_vs_smas'] = self._price_position_numeric(df['close'], df['sma_10'], df['sma_20'], df['sma_50'])
        
        # 8. Distance Strength - from SMASimpleStrategy
        df['price_dist_sma10'] = (df['close'] - df['sma_10']) / df['sma_10']
        df['price_dist_sma20'] = (df['close'] - df['sma_20']) / df['sma_20']
        df['price_dist_sma50'] = (df['close'] - df['sma_50']) / df['sma_50']
        
        # 9. Trend Alignment (all SMAs in same direction)
        df['sma_trend_alignment'] = self._calculate_trend_alignment(
            df['sma_10_slope'], df['sma_20_slope'], df['sma_50_slope']
        )
        
        # 10. SMA Convergence/Divergence
        df['sma_convergence_10_20'] = self._calculate_convergence(df['sma_10'], df['sma_20'])
        df['sma_convergence_20_50'] = self._calculate_convergence(df['sma_20'], df['sma_50'])
        
        # 11. Lags of main features (6 periods as in book)
        base_features = [
            'return', 'momentum', 'volatility', 'volume_ratio', 'price_norm',
            'cross_strength_10_20', 'cross_strength_20_50', 'price_vs_smas',
            'sma_trend_alignment'
        ]
        
        for feature in base_features:
            if feature in df.columns:
                for lag in range(1, self.config.lags + 1):
                    df[f'{feature}_lag_{lag}'] = df[feature].shift(lag)
        
        return df
    
    def _calculate_slope(self, series: pd.Series) -> pd.Series:
        """
        Calculate slope/trend of a series (copied from live_trading_system).
        """
        slopes = pd.Series(index=series.index, dtype=float)
        
        for i in range(3, len(series)):
            recent_values = series.iloc[i-2:i+1]  # Last 3 points
            if len(recent_values.dropna()) >= 2:
                slope = (recent_values.iloc[-1] - recent_values.iloc[0]) / 2
                slopes.iloc[i] = slope
            else:
                slopes.iloc[i] = 0
        
        return slopes.fillna(0)
    
    def _detect_cross(self, fast_sma: pd.Series, slow_sma: pd.Series) -> pd.Series:
        """
        Detecta cruzamentos entre SMAs (copiado do SMACrossoverStrategy).
        
        Returns:
            +1: Golden Cross (fast cruza para cima)
            -1: Death Cross (fast cruza para baixo)
             0: Sem cruzamento
        """
        crosses = pd.Series(index=fast_sma.index, dtype=float).fillna(0)
        
        for i in range(1, len(fast_sma)):
            if pd.isna(fast_sma.iloc[i]) or pd.isna(slow_sma.iloc[i]):
                continue
            if pd.isna(fast_sma.iloc[i-1]) or pd.isna(slow_sma.iloc[i-1]):
                continue
                
            prev_fast, prev_slow = fast_sma.iloc[i-1], slow_sma.iloc[i-1]
            curr_fast, curr_slow = fast_sma.iloc[i], slow_sma.iloc[i]
            
            # Golden Cross
            if prev_fast <= prev_slow and curr_fast > curr_slow:
                crosses.iloc[i] = 1
            # Death Cross
            elif prev_fast >= prev_slow and curr_fast < curr_slow:
                crosses.iloc[i] = -1
        
        return crosses
    
    def _price_position_numeric(self, price: pd.Series, sma_10: pd.Series, 
                               sma_20: pd.Series, sma_50: pd.Series) -> pd.Series:
        """
        Posição numérica do preço relativa às SMAs.
        
        Returns:
            +2: Acima de todas as SMAs
            +1: Acima de algumas SMAs
             0: Entre SMAs
            -1: Abaixo de algumas SMAs
            -2: Abaixo de todas as SMAs
        """
        positions = pd.Series(index=price.index, dtype=float)
        
        for i in range(len(price)):
            if any(pd.isna(val) for val in [price.iloc[i], sma_10.iloc[i], sma_20.iloc[i], sma_50.iloc[i]]):
                positions.iloc[i] = 0
                continue
            
            p, s10, s20, s50 = price.iloc[i], sma_10.iloc[i], sma_20.iloc[i], sma_50.iloc[i]
            smas = [s10, s20, s50]
            
            if p > max(smas):
                positions.iloc[i] = 2  # Acima de todas
            elif p < min(smas):
                positions.iloc[i] = -2  # Abaixo de todas
            elif p > np.median(smas):
                positions.iloc[i] = 1   # Acima da maioria
            elif p < np.median(smas):
                positions.iloc[i] = -1  # Abaixo da maioria
            else:
                positions.iloc[i] = 0   # Entre
        
        return positions.fillna(0)
    
    def _calculate_trend_alignment(self, slope_10: pd.Series, slope_20: pd.Series, 
                                  slope_50: pd.Series) -> pd.Series:
        """
        Calcula alinhamento de tendências das SMAs.
        
        Returns:
            +1: Todas subindo
            -1: Todas descendo
             0: Divergentes
        """
        alignment = pd.Series(index=slope_10.index, dtype=float)
        
        for i in range(len(slope_10)):
            s10, s20, s50 = slope_10.iloc[i], slope_20.iloc[i], slope_50.iloc[i]
            
            if all(s > 0 for s in [s10, s20, s50]):
                alignment.iloc[i] = 1   # Todas subindo
            elif all(s < 0 for s in [s10, s20, s50]):
                alignment.iloc[i] = -1  # Todas descendo
            else:
                alignment.iloc[i] = 0   # Divergentes
        
        return alignment.fillna(0)
    
    def _calculate_convergence(self, fast_sma: pd.Series, slow_sma: pd.Series) -> pd.Series:
        """
        Calcula se SMAs estão convergindo ou divergindo.
        
        Returns:
            Positivo: Convergindo
            Negativo: Divergindo
        """
        diff = abs(fast_sma - slow_sma)
        convergence = -diff.diff()  # Negativo da derivada da diferença
        
        return convergence.fillna(0)
    
    def create_target(self, data: pd.DataFrame) -> pd.Series:
        """
        Criar target baseado no livro: direção do próximo movimento.
        """
        # Próximo retorno
        future_return = data['close'].shift(-1).pct_change(fill_method=None)
        
        # Direção: +1 para up, -1 para down (como no livro AdaBoost)
        target = np.where(future_return > 0, 1, -1)
        
        return pd.Series(target, index=data.index, name='direction')
    
    def prepare_ml_data(self, data: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
        """Preparar dados para ML como no livro."""
        # Criar features
        features_df = self.create_features_inline(data)
        
        # Criar target
        target = self.create_target(features_df)
        
        # Selecionar features para ML (excluir auxiliares)
        exclude_cols = {
            'direction', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
            'sma_10', 'sma_20', 'sma_50', 'price_min', 'price_max', 'volume_avg'
        }
        
        feature_cols = [col for col in features_df.columns if col not in exclude_cols]
        
        # Preparar X e y
        X = features_df[feature_cols]
        y = target
        
        # Remover NaNs
        valid_mask = ~(X.isna().any(axis=1) | y.isna())
        X = X[valid_mask]
        y = y[valid_mask]
        
        self.feature_columns = feature_cols
        self.logger.info("ML data prepared", 
                        features=len(feature_cols), 
                        samples=len(X))
        
        return X, y
    
    def create_bagging_models(self) -> Tuple[BaggingRegressor, BaggingRegressor]:
        """Criar modelos BaggingRegressor conforme especificação."""
        
        # DecisionTree base
        tree_base = DecisionTreeRegressor(
            max_depth=5,
            min_samples_leaf=10,
            random_state=42
        )
        
        tree_model = BaggingRegressor(
            estimator=tree_base,
            n_estimators=100,
            max_samples=0.8,
            max_features=0.8,
            random_state=42,
            n_jobs=-1
        )
        
        # MLP base
        mlp_base = MLPRegressor(
            hidden_layer_sizes=(50, 25),
            activation='relu',
            solver='adam',
            alpha=0.01,
            max_iter=1000,
            early_stopping=True,
            random_state=42
        )
        
        mlp_model = BaggingRegressor(
            estimator=mlp_base,
            n_estimators=20,  # Menos para MLP (mais lento)
            max_samples=0.9,
            random_state=42,
            n_jobs=2
        )
        
        return tree_model, mlp_model
    
    def train_once(self, historical_data: pd.DataFrame) -> Dict[str, Any]:
        """
        Treinamento único como no livro (sem retraining).
        
        Split sequencial: 70% treino, 30% teste (como no livro página 301).
        """
        try:
            self.logger.info("Starting single training session")
            
            # Preparar dados ML
            X, y = self.prepare_ml_data(historical_data)
            
            if len(X) < 100:
                error_msg = f"Dados insuficientes: {len(X)} < 100"
                self.logger.error(error_msg)
                return {'status': 'failed', 'error': error_msg}
            
            # Split sequencial como no livro (70% treino)
            split = int(len(X) * 0.7)
            X_train, X_test = X.iloc[:split], X.iloc[split:]
            y_train, y_test = y.iloc[:split], y.iloc[split:]
            
            self.logger.info("Data split", 
                           train_samples=len(X_train), 
                           test_samples=len(X_test))
            
            # Check and handle NaN values
            self.logger.info(f"Checking for NaN values in training data...")
            nan_mask = pd.isna(X_train).any(axis=1) | pd.isna(y_train)
            if nan_mask.sum() > 0:
                self.logger.warning(f"Found {nan_mask.sum()} rows with NaN values, removing them")
                X_train = X_train[~nan_mask]
                y_train = y_train[~nan_mask]
            
            nan_mask_test = pd.isna(X_test).any(axis=1)
            if nan_mask_test.sum() > 0:
                self.logger.warning(f"Found {nan_mask_test.sum()} rows with NaN values in test set, removing them")
                X_test = X_test[~nan_mask_test]
                y_test = y_test[~nan_mask_test]
            
            # Normalização (como no livro)
            X_train_scaled = self.scaler.fit_transform(X_train)
            X_test_scaled = self.scaler.transform(X_test)
            
            # Criar e treinar modelos
            self.tree_model, self.mlp_model = self.create_bagging_models()
            
            self.logger.info("Training DecisionTree BaggingRegressor")
            self.tree_model.fit(X_train_scaled, y_train)
            
            self.logger.info("Training MLP BaggingRegressor")
            self.mlp_model.fit(X_train_scaled, y_train)
            
            # Avaliar modelos
            tree_pred = self.tree_model.predict(X_test_scaled)
            mlp_pred = self.mlp_model.predict(X_test_scaled)
            
            tree_r2 = r2_score(y_test, tree_pred)
            mlp_r2 = r2_score(y_test, mlp_pred)
            
            # Salvar métricas
            self.model_metrics = {
                'tree_r2': tree_r2,
                'mlp_r2': mlp_r2,
                'n_samples': len(X),
                'n_features': X.shape[1],
                'train_samples': len(X_train),
                'test_samples': len(X_test),
                'training_date': datetime.now(),
                'feature_columns': self.feature_columns
            }
            
            self.is_trained = True
            
            # Salvar modelo (como no livro - pickle)
            self.save_model()
            
            self.logger.info("Training completed successfully",
                           tree_r2=tree_r2,
                           mlp_r2=mlp_r2,
                           features=X.shape[1])
            
            return {
                'status': 'success',
                'metrics': self.model_metrics
            }
            
        except Exception as e:
            error_msg = f"Erro no treinamento: {str(e)}"
            self.logger.error(error_msg)
            return {'status': 'failed', 'error': error_msg}
    
    def predict_ensemble(self, current_data: pd.DataFrame) -> Tuple[float, float, float, float]:
        """
        Predição ensemble: 60% Tree + 40% MLP (como configurado).
        """
        try:
            if not self.is_trained:
                return 0.0, 0.0, 0.0, 0.0
            
            # Preparar features
            features_df = self.create_features_inline(current_data)
            X_pred = features_df[self.feature_columns].tail(1).ffill()
            
            # Check for NaN values in prediction data
            if X_pred.isna().any().any():
                self.logger.warning("NaN values found in prediction features, using forward fill")
                X_pred = X_pred.ffill().bfill()  # ffill + bfill as fallback
                
                # If still NaN after ffill/bfill, use mean from training data
                if X_pred.isna().any().any():
                    self.logger.warning("Still have NaN after ffill/bfill, using feature means")
                    feature_means = pd.Series(self.scaler.mean_, index=X_pred.columns)
                    X_pred = X_pred.fillna(feature_means)
            
            X_pred_scaled = self.scaler.transform(X_pred)
            
            # Predições individuais
            tree_pred = self.tree_model.predict(X_pred_scaled)[0]
            mlp_pred = self.mlp_model.predict(X_pred_scaled)[0]
            
            # Ensemble final (60% tree, 40% mlp)
            final_pred = (tree_pred * self.config.tree_weight + 
                         mlp_pred * self.config.mlp_weight)
            
            # Confiança baseada na concordância
            agreement = 1 - abs(tree_pred - mlp_pred) / 2
            confidence = min(abs(agreement), 1.0)
            
            return final_pred, tree_pred, mlp_pred, confidence
            
        except Exception as e:
            self.logger.error("Prediction error", error=str(e))
            return 0.0, 0.0, 0.0, 0.0
    
    def generate_signal(self, current_data: pd.DataFrame) -> Optional[Dict[str, Any]]:
        """
        Gerar sinal de trading baseado na predição.
        """
        if not self.is_trained:
            return None
        
        final_pred, tree_pred, mlp_pred, confidence = self.predict_ensemble(current_data)
        
        # Verificar threshold de confiança
        if confidence < self.config.confidence_threshold:
            return None
        
        # Determinar direção
        signal_type = "buy" if final_pred > 0 else "sell"
        current_price = current_data['close'].iloc[-1]
        
        signal = {
            'timestamp': pd.Timestamp.now(),
            'asset_symbol': self.config.asset_symbol,
            'signal_type': signal_type,
            'price': current_price,
            'confidence': confidence,
            'predictions': {
                'tree': tree_pred,
                'mlp': mlp_pred,
                'final': final_pred
            },
            'reason': f"ML Ensemble: Tree={tree_pred:.3f}, MLP={mlp_pred:.3f}, Final={final_pred:.3f}"
        }
        
        self.logger.info("Signal generated",
                        signal_type=signal_type,
                        confidence=confidence,
                        prediction=final_pred)
        
        return signal
    
    def save_model(self):
        """Salvar modelo como no livro (pickle)."""
        try:
            model_data = {
                'tree_model': self.tree_model,
                'mlp_model': self.mlp_model,
                'scaler': self.scaler,
                'feature_columns': self.feature_columns,
                'model_metrics': self.model_metrics,
                'config': self.config
            }
            
            with open(self.config.model_path, 'wb') as f:
                pickle.dump(model_data, f)
                
            self.logger.info("Model saved", path=self.config.model_path)
            
        except Exception as e:
            self.logger.error("Error saving model", error=str(e))
    
    def load_model(self) -> bool:
        """Carregar modelo salvo."""
        try:
            if not Path(self.config.model_path).exists():
                return False
            
            with open(self.config.model_path, 'rb') as f:
                model_data = pickle.load(f)
            
            self.tree_model = model_data['tree_model']
            self.mlp_model = model_data['mlp_model']
            self.scaler = model_data['scaler']
            self.feature_columns = model_data['feature_columns']
            self.model_metrics = model_data['model_metrics']
            
            self.is_trained = True
            
            self.logger.info("Model loaded successfully", 
                           features=len(self.feature_columns))
            return True
            
        except Exception as e:
            self.logger.error("Error loading model", error=str(e))
            return False




In [ ]:
#!/usr/bin/env python3
"""
SOL-BRL Data Collection for Trading Exercise
==========================================

Collect exactly 2,016 bars (1 week of 5min data) for SOL-BRL.
Based on candle_downloader.py from practice_cpf_program.

Author: Everaldo Gomes  
Date: 2025-08-08
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
import sys
import sqlite3
from typing import Optional

# Add mercado-bitcoin-python to path
sys.path.append('/home/everaldo/Code/mercado-bitcoin-python')
from mercado_bitcoin_python import MercadoBitcoinClient
from mercado_bitcoin_python.api.client import MBConfig

class SOLDataCollector:
    """Simplified data collector for SOL-BRL exercise data."""
    
    def __init__(self, api_config: dict = None):
        """
        Initialize collector.
        
        Args:
            api_config: Dict with api_key and api_secret (optional)
        """
        self.asset_symbol = "SOL-BRL"
        self.timeframe = "5m"
        self.target_bars = 2016  # 1 week = 5min * 12/h * 24h * 7d
        
        # Data storage
        self.data_path = Path("data/exercise")
        self.data_path.mkdir(parents=True, exist_ok=True)
        
        # Initialize API (if config available)
        self.api = None
        if api_config and api_config.get('api_key') and api_config.get('api_secret'):
            try:
                # Create MBConfig
                config = MBConfig(
                    api_key=api_config['api_key'],
                    api_secret=api_config['api_secret'],
                    api_key_type='read_only',
                    safe_mode=True,
                    enable_cache=True
                )
                
                self.api = MercadoBitcoinClient(config)
                print(f"✅ Connected to Mercado Bitcoin API")
            except Exception as e:
                print(f"⚠️  API connection failed: {e}")
                print("Will create synthetic data instead")
        else:
            print("ℹ️  No API config provided, will create synthetic data")
    
    def calculate_date_range(self) -> tuple[datetime, datetime]:
        """
        Calculate start/end dates for exactly 2,016 bars.
        From current time (2025-08-08) going back 1 week.
        """
        end_date = datetime(2025, 8, 8, 12, 0)  # Today at noon
        start_date = end_date - timedelta(minutes=5 * self.target_bars)
        
        print(f"📅 Target period: {start_date} → {end_date}")
        print(f"📊 Target bars: {self.target_bars} (5min intervals)")
        
        return start_date, end_date
    
    def collect_real_data(self) -> Optional[pd.DataFrame]:
        """Collect real SOL-BRL data from API."""
        if not self.api:
            return None
            
        try:
            start_date, end_date = self.calculate_date_range()
            
            print(f"🔄 Collecting {self.asset_symbol} data...")
            print(f"   Timeframe: {self.timeframe}")
            print(f"   Period: {start_date.strftime('%Y-%m-%d %H:%M')} to {end_date.strftime('%Y-%m-%d %H:%M')}")
            
            # Use get_candles method (native API)
            data = self.api.get_candles(
                symbol=self.asset_symbol,
                timeframe=self.timeframe,
                start_date=start_date,
                end_date=end_date
            )
            
            if data is None or data.empty:
                print("❌ No data returned from API")
                return None
            
            # Standardize column names
            if 'timestamp' in data.columns and 'timestamp' != data.index.name:
                data.set_index('timestamp', inplace=True)
            
            # Ensure we have OHLCV columns
            required_cols = ['open', 'high', 'low', 'close', 'volume']
            missing_cols = [col for col in required_cols if col not in data.columns]
            
            if missing_cols:
                print(f"❌ Missing columns: {missing_cols}")
                return None
            
            print(f"✅ Collected {len(data)} bars of real {self.asset_symbol} data")
            print(f"   Date range: {data.index[0]} → {data.index[-1]}")
            print(f"   Price range: {data['close'].min():.2f} - {data['close'].max():.2f} BRL")
            
            return data
            
        except Exception as e:
            print(f"❌ Error collecting real data: {e}")
            return None
    
    def create_synthetic_data(self) -> pd.DataFrame:
        """Create synthetic SOL-BRL data as fallback."""
        print("🎭 Creating synthetic SOL-BRL data...")
        
        start_date, end_date = self.calculate_date_range()
        
        # Create time index
        dates = pd.date_range(start_date, end_date, freq='5min')[:-1]  # Remove last to get exact count
        
        # SOL realistic price movement (around 150 BRL)
        base_price = 150.0
        np.random.seed(42)  # Reproducible
        
        # Generate more realistic price movement
        n = len(dates)
        
        # Trend component (gradual increase)
        trend = np.linspace(0, 0.3, n)  # 30% increase over week
        
        # Random walk component  
        random_walk = np.cumsum(np.random.normal(0, 0.002, n))  # 0.2% std per 5min
        
        # Cyclical component (daily patterns)
        daily_cycle = np.sin(2 * np.pi * np.arange(n) / (12 * 24)) * 0.05  # 5% daily oscillation
        
        # Combine components
        log_prices = np.log(base_price) + trend + random_walk + daily_cycle
        prices = np.exp(log_prices)
        
        # Generate OHLC with realistic spreads
        spread = 0.001  # 0.1% typical spread
        
        opens = prices * (1 + np.random.normal(0, spread/2, n))
        highs = prices * (1 + np.abs(np.random.normal(0, spread, n)))
        lows = prices * (1 - np.abs(np.random.normal(0, spread, n)))
        closes = prices
        
        # Ensure OHLC logic: low <= open,close <= high
        lows = np.minimum(lows, np.minimum(opens, closes))
        highs = np.maximum(highs, np.maximum(opens, closes))
        
        # Generate volume (correlated with volatility)
        volatility = np.abs(np.diff(log_prices, prepend=log_prices[0]))
        volumes = 1000 + volatility * 100000 + np.random.exponential(2000, n)
        
        data = pd.DataFrame({
            'open': opens,
            'high': highs,
            'low': lows,
            'close': closes,
            'volume': volumes.astype(int)
        }, index=dates)
        
        data.index.name = 'timestamp'
        
        print(f"✅ Created {len(data)} synthetic bars")
        print(f"   Date range: {data.index[0]} → {data.index[-1]}")
        print(f"   Price range: {data['close'].min():.2f} - {data['close'].max():.2f} BRL")
        print(f"   Volume range: {data['volume'].min():,} - {data['volume'].max():,}")
        
        return data
    
    def save_to_sqlite(self, data: pd.DataFrame) -> Path:
        """Save data to SQLite database."""
        db_path = self.data_path / "sol_data.db"
        
        with sqlite3.connect(db_path) as conn:
            # Save data
            data.to_sql("sol_bars", conn, if_exists='replace', index=True)
            
            # Create index for fast timestamp queries
            conn.execute("CREATE INDEX IF NOT EXISTS idx_timestamp ON sol_bars (timestamp)")
            
            # Add metadata table
            metadata = pd.DataFrame([{
                'asset_symbol': self.asset_symbol,
                'timeframe': self.timeframe,
                'total_bars': len(data),
                'start_date': str(data.index[0]),
                'end_date': str(data.index[-1]),
                'collection_date': str(datetime.now()),
                'data_type': 'real' if self.api else 'synthetic'
            }])
            
            metadata.to_sql("metadata", conn, if_exists='replace', index=False)
        
        print(f"💾 Data saved to: {db_path}")
        return db_path
    
    def save_to_csv(self, data: pd.DataFrame) -> Path:
        """Save data to CSV as backup."""
        csv_path = self.data_path / f"{self.asset_symbol.replace('-', '_')}_{self.timeframe}.csv"
        data.to_csv(csv_path)
        print(f"💾 CSV backup saved to: {csv_path}")
        return csv_path
    
    def collect_and_save(self) -> tuple[pd.DataFrame, Path]:
        """Main collection method."""
        print("=" * 60)
        print("SOL-BRL Data Collection for Trading Exercise")
        print("=" * 60)
        
        # Try to collect real data first
        data = self.collect_real_data()
        
        # Fallback to synthetic if real data fails
        if data is None:
            data = self.create_synthetic_data()
        
        # Validate data
        if len(data) != self.target_bars:
            print(f"⚠️  Got {len(data)} bars, expected {self.target_bars}")
            if len(data) < self.target_bars:
                print("   Using available data (should be sufficient for exercise)")
        
        # Save data
        db_path = self.save_to_sqlite(data)
        csv_path = self.save_to_csv(data)
        
        # Data quality summary
        print("\n📊 Data Quality Summary:")
        print(f"   Total bars: {len(data):,}")
        print(f"   Missing values: {data.isnull().sum().sum()}")
        print(f"   Date range: {(data.index[-1] - data.index[0]).days} days")
        print(f"   Price volatility: {(data['close'].std() / data['close'].mean() * 100):.2f}%")
        print(f"   Average volume: {data['volume'].mean():,.0f}")
        
        print("\n✅ Data collection completed!")
        print(f"Ready for ML training with {len(data)} bars")
        
        return data, db_path

def load_collected_data(db_path: str = "data/exercise/sol_data.db") -> pd.DataFrame:
    """Load previously collected data."""
    with sqlite3.connect(db_path) as conn:
        data = pd.read_sql("SELECT * FROM sol_bars", conn, index_col='timestamp', parse_dates=['timestamp'])
    
    print(f"📈 Loaded {len(data)} bars from {db_path}")
    return data


In [2]:
# Configure structured logging
def configure_logging():
    """Configure logging: traditional format for console/file."""
    from pathlib import Path
    import logging as std_logging
    from loguru import logger as loguru_logger
    import sys
    
    # Create logs directory
    logs_dir = Path("logs")
    logs_dir.mkdir(exist_ok=True)
    
    # Traditional format for console and main log file
    traditional_format = "%(asctime)s [%(levelname)-8s] %(message)s"
    
    # Configure standard logging - BOTH console and file  
    file_handler = std_logging.FileHandler(logs_dir / "intelligent_trading.log")
    file_handler.setFormatter(std_logging.Formatter(traditional_format, datefmt="%Y-%m-%d %H:%M:%S"))
    file_handler.setLevel(std_logging.DEBUG)  # Keep all levels in file
    
    console_handler = std_logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(std_logging.Formatter(traditional_format, datefmt="%Y-%m-%d %H:%M:%S"))
    console_handler.setLevel(std_logging.INFO)  # Only INFO+ in console
    
    root_logger = std_logging.getLogger()
    root_logger.setLevel(std_logging.DEBUG)
    root_logger.addHandler(file_handler)
    root_logger.addHandler(console_handler)
    
    # Configure loguru (used by mercado_bitcoin_python) to also log to our files
    # Keep default handler for nice colored console output - set to INFO level
    loguru_logger.configure(handlers=[
        {"sink": sys.stdout, "level": "INFO", "colorize": True},  # Console: INFO+ only
        {"sink": logs_dir / "intelligent_trading.log", "level": "DEBUG", "format": "{time:YYYY-MM-DD HH:mm:ss} [{level:<8}] {message}"}  # File: all levels
    ])
    
    # Configure structlog - colorful console renderer (no JSON)
    structlog.configure(
        processors=[
            structlog.dev.ConsoleRenderer(colors=True)  # Colorful format like loguru, no JSON
        ],
        context_class=dict,
        logger_factory=structlog.PrintLoggerFactory(),  # Direct printing
        cache_logger_on_first_use=True,
    )

configure_logging()
logging = structlog.get_logger()

In [3]:
@dataclass
class TradingPosition:
    """Track individual trading position."""
    symbol: str
    side: str  # 'buy' or 'sell'
    quantity: float
    entry_price: float
    entry_time: datetime
    fees_paid: float
    order_id: str
    
    @property
    def entry_value(self) -> float:
        """Total value invested including fees."""
        return (self.quantity * self.entry_price) + self.fees_paid
    
    def calculate_breakeven_price(self, exit_fees_pct: float) -> float:
        """Calculate minimum price to breakeven on exit."""
        # Need to recover entry value after paying exit fees
        return self.entry_value / (self.quantity * (1 - exit_fees_pct / 100))
    
    def calculate_current_pnl(self, current_price: float, exit_fees_pct: float) -> float:
        """Calculate current P&L including exit fees."""
        gross_exit_value = self.quantity * current_price
        exit_fees = gross_exit_value * (exit_fees_pct / 100)
        net_exit_value = gross_exit_value - exit_fees
        
        return net_exit_value - self.entry_value

In [4]:
@dataclass
class TradingSession:
    """Track complete trading session."""
    session_id: str
    start_time: datetime
    end_time: Optional[datetime] = None
    initial_balance: float = 0.0
    final_balance: float = 0.0
    total_trades: int = 0
    winning_trades: int = 0
    losing_trades: int = 0
    total_fees_paid: float = 0.0
    positions: List[TradingPosition] = None
    
    def __post_init__(self):
        if self.positions is None:
            self.positions = []
    
    @property
    def win_rate(self) -> float:
        if self.total_trades == 0:
            return 0.0
        return (self.winning_trades / self.total_trades) * 100
    
    @property
    def net_pnl(self) -> float:
        return self.final_balance - self.initial_balance
    
    @property
    def roi_pct(self) -> float:
        if self.initial_balance == 0:
            return 0.0
        return (self.net_pnl / self.initial_balance) * 100
    
    @property
    def duration_hours(self) -> float:
        if not self.end_time:
            return 0.0
        return (self.end_time - self.start_time).total_seconds() / 3600

In [5]:
class SimpleDatabase:
    """Simple SQLite database for session tracking."""
    
    def __init__(self, db_path: str):
        self.db_path = db_path
        Path(db_path).parent.mkdir(parents=True, exist_ok=True)
        self._create_tables()
    
    def _create_tables(self):
        """Create necessary tables."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("""
                CREATE TABLE IF NOT EXISTS sessions (
                    session_id TEXT PRIMARY KEY,
                    start_time TEXT,
                    end_time TEXT,
                    initial_balance REAL,
                    final_balance REAL,
                    total_trades INTEGER,
                    winning_trades INTEGER,
                    losing_trades INTEGER,
                    total_fees_paid REAL,
                    created_at TEXT DEFAULT CURRENT_TIMESTAMP
                )
            """)
            
            conn.execute("""
                CREATE TABLE IF NOT EXISTS positions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    session_id TEXT,
                    symbol TEXT,
                    side TEXT,
                    quantity REAL,
                    entry_price REAL,
                    entry_time TEXT,
                    fees_paid REAL,
                    order_id TEXT,
                    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (session_id) REFERENCES sessions (session_id)
                )
            """)
    
    def save_session(self, session: TradingSession):
        """Save session to database."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("""
                INSERT OR REPLACE INTO sessions 
                (session_id, start_time, end_time, initial_balance, final_balance,
                 total_trades, winning_trades, losing_trades, total_fees_paid)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                session.session_id,
                session.start_time.isoformat(),
                session.end_time.isoformat() if session.end_time else None,
                session.initial_balance,
                session.final_balance,
                session.total_trades,
                session.winning_trades,
                session.losing_trades,
                session.total_fees_paid
            ))

In [6]:
class IntelligentTradingSystem:
    """Main intelligent trading system with anti-loss protection."""
    
    def __init__(self, config: TradingConfig):
        self.config = config
        self.logger = logging.bind(system="intelligent_trading")
        
        # Create directories
        config.create_directories()
        
        # Initialize components
        ml_config = MLTradingConfig(
            asset_symbol=config.asset_symbol,
            total_budget=config.total_budget,
            max_positions=config.max_positions,
            position_size=config.position_size
        )
        self.strategy = SimpleMLEnsembleStrategy(ml_config)
        self.telegram = TelegramService(config)
        self.database = SimpleDatabase(config.database_path)
        
        # Initialize real API client
        from mercado_bitcoin_python.api.client import MercadoBitcoinClient, MBConfig
        
        self.api = None
        self.api_client = None
        
        if config.mb_api_key and config.mb_api_secret:
            try:
                # Initialize the real Mercado Bitcoin client
                mb_config = MBConfig(
                    api_key=config.mb_api_key,
                    api_secret=config.mb_api_secret,
                    api_key_type="read_write" if not config.mb_read_only else "read_only",
                    safe_mode=config.mb_read_only,
                    enable_cache=True,
                    cache_dir=f"{config.data_path}/cache"
                )
                
                self.api_client = MercadoBitcoinClient(mb_config)
                self.logger.info("Real API client initialized",
                               safe_mode=mb_config.safe_mode,
                               api_type=mb_config.api_key_type)
                
                # Also create MBTpqoa for compatibility (if needed for some legacy calls)
                try:
                    self.api = MBTpqoa(
                        api_key=config.mb_api_key,
                        api_secret=config.mb_api_secret,
                        read_only=config.mb_read_only
                    )
                except Exception as mbapi_error:
                    self.logger.warning("MBTpqoa initialization failed, using only MercadoBitcoinClient", error=str(mbapi_error))
                    self.api = None
                
            except Exception as e:
                self.logger.error("API initialization failed", error=str(e))
        
        # Trading state
        self.account_id: Optional[str] = None
        self.account_summary: Optional[Dict[str, Any]] = None
        self.active_positions: List[TradingPosition] = []
        self.session: Optional[TradingSession] = None
        self.market_data: Optional[pd.DataFrame] = None
    
    async def initialize_session(self):
        """Initialize trading session with account info collection."""
        self.logger.info("Initializing intelligent trading session")
        
        session_id = f"session_{int(datetime.now().timestamp())}"
        self.session = TradingSession(
            session_id=session_id,
            start_time=datetime.now()
        )
        
        if self.api_client:
            try:
                # Get account information using real API client
                accounts = self.api_client.get_accounts()
                if accounts:
                    self.account_id = accounts[0].account_id
                    self.logger.info("Account ID obtained", account_id=self.account_id[:8] + "...")
                
                # Get comprehensive account summary
                self.account_summary = self.api_client.get_account_summary(
                    self.account_id, 
                    self.config.asset_symbol
                )
                
                # Store initial balance
                self.session.initial_balance = self.account_summary['balances']['BRL']
                
                self.logger.info("Account summary collected",
                               tier=self.account_summary['tier'],
                               maker_fee=self.account_summary['fees']['maker'],
                               taker_fee=self.account_summary['fees']['taker'])
                
            except Exception as e:
                self.logger.error("Failed to collect account info", error=str(e))
                # Create mock summary for testing
                self.account_summary = {
                    'account_id': 'test_account',
                    'tier': 'standard',
                    'symbol': self.config.asset_symbol,
                    'fees': {'maker': 0.25, 'taker': 0.35, 'total_cycle': 1.2},
                    'balances': {'SOL': 0.0, 'BRL': 100.0},
                    'positions': 0
                }
                self.session.initial_balance = 100.0
        else:
            # Mock data for test mode
            self.account_summary = {
                'account_id': 'test_account',
                'tier': 'standard',
                'symbol': self.config.asset_symbol,
                'fees': {'maker': 0.25, 'taker': 0.35, 'total_cycle': 1.2},
                'balances': {'SOL': 0.0, 'BRL': 100.0},
                'positions': 0
            }
            self.session.initial_balance = 100.0
        
        # Load market data and train strategy
        await self._prepare_strategy()
        
        # Print session info
        self._print_session_info()
        
        # Send Telegram notification
        session_info = {
            'start_time': self.session.start_time,
            'session_id': self.session.session_id
        }
        await self.telegram.send_session_start(self.account_summary, session_info)
    
    def _print_session_info(self):
        """Print comprehensive session information."""
        print("=" * 80)
        print("🤖 INTELLIGENT TRADING SYSTEM - SESSION START")
        print("=" * 80)
        print(f"📊 Account Information:")
        print(f"   Account ID: {self.account_summary['account_id'][:12]}...")
        print(f"   Tier: {self.account_summary['tier']}")
        print(f"   Asset: {self.account_summary['symbol']}")
        
        print(f"\n💰 Trading Fees ({self.account_summary['symbol']}):")
        print(f"   Maker Fee: {self.account_summary['fees']['maker']:.4f}%")
        print(f"   Taker Fee: {self.account_summary['fees']['taker']:.4f}%")
        print(f"   Total Cycle (Buy+Sell): {self.account_summary['fees']['total_cycle']:.4f}%")
        
        print(f"\n💵 Initial Balances:")
        print(f"   SOL: {self.account_summary['balances']['SOL']:.6f}")
        print(f"   BRL: R$ {self.account_summary['balances']['BRL']:.2f}")
        
        print(f"\n🎯 Session Configuration:")
        print(f"   Total Budget: R$ {self.config.total_budget:.2f}")
        print(f"   Max Positions: {self.config.max_positions}")
        print(f"   Position Size: R$ {self.config.position_size:.2f}")
        print(f"   Anti-Loss Protection: ENABLED")
        
        print(f"\n⏰ Session Started: {self.session.start_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 80)
    
    async def _prepare_strategy(self):
        """Load fresh data and prepare ML strategy."""
        self.logger.info("Preparing ML strategy with fresh data")
        
        # Step 1: Update cache with latest data (last week)
        await self._update_market_data_cache()
        
        # Step 2: Load updated market data
        try:
            self.market_data = load_collected_data(f"{self.config.data_path}/exercise/sol_data.db")
            self.logger.info(f"Loaded {len(self.market_data)} bars from updated cache")
        except Exception as e:
            self.logger.warning("Could not load collected data", error=str(e))
            # Create sample data for testing
            dates = pd.date_range(start='2025-08-01', periods=2016, freq='5T')
            self.market_data = pd.DataFrame({
                'timestamp': dates,
                'open': 180.0 + (pd.Series(range(2016)) * 0.01),
                'high': 185.0 + (pd.Series(range(2016)) * 0.01),
                'low': 175.0 + (pd.Series(range(2016)) * 0.01),
                'close': 180.0 + (pd.Series(range(2016)) * 0.01),
                'volume': 1000.0
            })
        
        # Step 3: Always retrain with fresh data
        self.logger.info("Training ML model with fresh weekly data")
        result = self.strategy.train_once(self.market_data)
        if result['status'] != 'success':
            raise Exception(f"Strategy training failed: {result.get('error')}")
        
        self.logger.info("ML strategy ready", 
                        is_trained=self.strategy.is_trained,
                        features=len(self.strategy.feature_columns) if hasattr(self.strategy, 'feature_columns') else 0)
    
    async def _update_market_data_cache(self):
        """Update market data cache with latest candles."""
        try:
            from collect_sol_data import SOLDataCollector
            
            self.logger.info("Updating market data cache...")
            
            # Pass API configuration to collector
            api_config = {
                'api_key': self.config.mb_api_key,
                'api_secret': self.config.mb_api_secret
            } if self.config.mb_api_key and self.config.mb_api_secret else None
            
            collector = SOLDataCollector(api_config)
            
            # Collect fresh data for the last week
            collector.collect_and_save()
            self.logger.info("Market data cache updated successfully")
            
        except Exception as e:
            self.logger.warning("Failed to update cache, using existing data", error=str(e))
    
    def _is_profitable_to_sell(self, position: TradingPosition, current_price: float) -> bool:
        """Check if selling would be profitable considering fees."""
        if not self.account_summary:
            return True  # Allow if no fee info
        
        exit_fee_pct = self.account_summary['fees']['maker']  # Assume maker fee for limit orders
        current_pnl = position.calculate_current_pnl(current_price, exit_fee_pct)
        
        self.logger.info("Profitability check",
                        entry_price=position.entry_price,
                        current_price=current_price,
                        breakeven_price=position.calculate_breakeven_price(exit_fee_pct),
                        current_pnl=current_pnl,
                        is_profitable=current_pnl > 0)
        
        return current_pnl > 0
    
    async def process_signal(self, signal: Dict[str, Any]) -> bool:
        """Process trading signal with anti-loss protection."""
        signal_type = signal['signal_type']
        current_price = signal['price']
        
        self.logger.info("Processing signal", 
                        type=signal_type,
                        price=current_price,
                        confidence=signal['confidence'])
        
        if signal_type == 'buy':
            return await self._process_buy_signal(signal)
        elif signal_type == 'sell':
            return await self._process_sell_signal(signal)
        
        return False
    
    async def _process_buy_signal(self, signal: Dict[str, Any]) -> bool:
        """Process buy signal."""
        # Check if we can open new position
        if len(self.active_positions) >= self.config.max_positions:
            self.logger.warning("Cannot open new position - max positions reached")
            return False
        
        # Calculate position size
        position_value = self.config.position_size
        quantity = position_value / signal['price']
        
        position_info = {
            'position_size': position_value,
            'quantity': quantity
        }
        
        # Send Telegram alert
        await self.telegram.send_signal_alert(signal, position_info)
        
        # Execute trade (or simulate)
        trade_result = await self._execute_buy_order(signal['price'], quantity)
        
        if trade_result['status'] == 'filled':
            # Create position record
            position = TradingPosition(
                symbol=self.config.asset_symbol,
                side='buy',
                quantity=quantity,
                entry_price=trade_result['price'],
                entry_time=datetime.now(),
                fees_paid=trade_result['fees'],
                order_id=trade_result['order_id']
            )
            
            self.active_positions.append(position)
            self.session.total_trades += 1
            self.session.total_fees_paid += trade_result['fees']
            
            # Send execution notification
            await self.telegram.send_trade_execution(trade_result)
            
            self.logger.info("Buy position opened",
                           quantity=quantity,
                           price=trade_result['price'],
                           value=trade_result['value'])
            return True
        
        return False
    
    async def _process_sell_signal(self, signal: Dict[str, Any]) -> bool:
        """Process sell signal with anti-loss protection."""
        if not self.active_positions:
            self.logger.info("No positions to sell")
            return False
        
        # Find position to sell (FIFO)
        position = self.active_positions[0]
        current_price = signal['price']
        
        # ANTI-LOSS PROTECTION: Check profitability
        if not self._is_profitable_to_sell(position, current_price):
            self.logger.warning("ANTI-LOSS PROTECTION: Rejecting unprofitable sell signal",
                              entry_price=position.entry_price,
                              current_price=current_price)
            
            # Send notification about protection
            position_dict = {
                'entry_price': position.entry_price,
                'breakeven_price': position.calculate_breakeven_price(self.account_summary['fees']['maker'])
            }
            reason = f"Would lose R$ {position.calculate_breakeven_price(self.account_summary['fees']['maker']) - current_price:.2f} in fees"
            await self.telegram.send_protection_alert(position_dict, current_price, reason)
            
            return False
        
        # Send signal alert
        position_info = {
            'position_size': position.quantity * current_price,
            'quantity': position.quantity
        }
        await self.telegram.send_signal_alert(signal, position_info)
        
        # Execute sell
        trade_result = await self._execute_sell_order(current_price, position.quantity)
        
        if trade_result['status'] == 'filled':
            # Calculate P&L
            pnl = position.calculate_current_pnl(
                trade_result['price'], 
                self.account_summary['fees']['maker']
            )
            
            # Update session stats
            self.session.total_trades += 1
            self.session.total_fees_paid += trade_result['fees']
            
            if pnl > 0:
                self.session.winning_trades += 1
            else:
                self.session.losing_trades += 1
            
            # Remove position
            self.active_positions.remove(position)
            
            # Send execution notification
            await self.telegram.send_trade_execution(trade_result, is_profitable=pnl > 0)
            
            self.logger.info("Sell position closed",
                           quantity=position.quantity,
                           entry_price=position.entry_price,
                           exit_price=trade_result['price'],
                           pnl=pnl)
            return True
        
        return False
    
    async def _execute_buy_order(self, price: float, quantity: float) -> Dict[str, Any]:
        """Execute buy order (real or simulated)."""
        if self.api_client and self.account_id and not self.config.mode == "test":
            try:
                # Use the real MercadoBitcoinClient
                result = self.api_client.place_market_order(
                    account_id=self.account_id,
                    symbol=self.config.asset_symbol,
                    side='buy',
                    quantity=quantity,
                    dry_run=self.config.mb_read_only
                )
                
                # Convert API response to expected format
                return {
                    'order_id': result.get('orderId', result.get('order_id', f'api_{int(datetime.now().timestamp())}')),
                    'status': 'filled',
                    'side': 'buy',
                    'price': price,  # Market price
                    'quantity': quantity,
                    'value': price * quantity,
                    'fees': (price * quantity) * (self.account_summary['fees']['taker'] / 100)
                }
            except Exception as e:
                self.logger.error("Real buy order failed", error=str(e))
        
        # Simulate order for test mode
        fees = (price * quantity) * (self.account_summary['fees']['taker'] / 100)
        return {
            'order_id': f'sim_buy_{int(datetime.now().timestamp())}',
            'status': 'filled',
            'side': 'buy',
            'price': price,
            'quantity': quantity,
            'value': price * quantity,
            'fees': fees
        }
    
    async def _execute_sell_order(self, price: float, quantity: float) -> Dict[str, Any]:
        """Execute sell order (real or simulated)."""
        if self.api_client and self.account_id and not self.config.mode == "test":
            try:
                # Use the real MercadoBitcoinClient
                result = self.api_client.place_market_order(
                    account_id=self.account_id,
                    symbol=self.config.asset_symbol,
                    side='sell',
                    quantity=quantity,
                    dry_run=self.config.mb_read_only
                )
                
                # Convert API response to expected format
                gross_value = price * quantity
                fees = gross_value * (self.account_summary['fees']['maker'] / 100)
                
                return {
                    'order_id': result.get('orderId', result.get('order_id', f'api_{int(datetime.now().timestamp())}')),
                    'status': 'filled',
                    'side': 'sell',
                    'price': price,  # Market price
                    'quantity': quantity,
                    'value': gross_value,
                    'fees': fees
                }
            except Exception as e:
                self.logger.error("Real sell order failed", error=str(e))
        
        # Simulate order for test mode
        gross_value = price * quantity
        fees = gross_value * (self.account_summary['fees']['maker'] / 100)
        
        return {
            'order_id': f'sim_sell_{int(datetime.now().timestamp())}',
            'status': 'filled',
            'side': 'sell',
            'price': price,
            'quantity': quantity,
            'value': gross_value,
            'fees': fees
        }
    
    async def finalize_session(self):
        """Finalize trading session."""
        if not self.session:
            return
        
        self.session.end_time = datetime.now()
        
        # Force close all positions if at session end to calculate final P&L
        if self.active_positions:
            self.logger.info("Force closing remaining positions")
            
            # Get current price for final valuation
            current_price = self.market_data['close'].iloc[-1] if self.market_data is not None else 200.0
            
            for position in self.active_positions:
                # Check if profitable to sell
                if self._is_profitable_to_sell(position, current_price):
                    trade_result = await self._execute_sell_order(current_price, position.quantity)
                    self.session.total_trades += 1
                    self.session.total_fees_paid += trade_result['fees']
                    
                    pnl = position.calculate_current_pnl(current_price, self.account_summary['fees']['maker'])
                    if pnl > 0:
                        self.session.winning_trades += 1
                    else:
                        self.session.losing_trades += 1
                else:
                    self.logger.warning("FINAL PROTECTION: Not selling losing position",
                                      symbol=position.symbol,
                                      entry_price=position.entry_price,
                                      current_price=current_price)
            
            self.active_positions.clear()
        
        # Calculate final balances (simulated)
        self.session.final_balance = self.session.initial_balance + self.session.net_pnl
        
        final_balances = {
            'SOL': sum(p.quantity for p in self.active_positions),  # Remaining SOL
            'BRL': self.session.final_balance
        }
        
        # Save session to database
        self.database.save_session(self.session)
        
        # Print final summary
        self._print_session_summary()
        
        # Send Telegram summary
        session_summary = {
            'duration_hours': self.session.duration_hours,
            'total_trades': self.session.total_trades,
            'win_rate': self.session.win_rate,
            'initial_balance': self.session.initial_balance,
            'final_balance': self.session.final_balance,
            'net_pnl': self.session.net_pnl,
            'roi_pct': self.session.roi_pct,
            'total_fees_paid': self.session.total_fees_paid,
            'end_time': self.session.end_time
        }
        await self.telegram.send_session_end(session_summary, final_balances)
        
        self.logger.info("Trading session finalized",
                        duration_hours=self.session.duration_hours,
                        total_trades=self.session.total_trades,
                        net_pnl=self.session.net_pnl)
    
    def _print_session_summary(self):
        """Print final session summary."""
        print("\n" + "=" * 80)
        print("🏁 INTELLIGENT TRADING SYSTEM - SESSION COMPLETE")
        print("=" * 80)
        print(f"📊 Session Performance:")
        print(f"   Duration: {self.session.duration_hours:.1f} hours")
        print(f"   Total Trades: {self.session.total_trades}")
        print(f"   Win Rate: {self.session.win_rate:.1f}%")
        
        print(f"\n💰 Financial Summary:")
        print(f"   Initial Balance: R$ {self.session.initial_balance:.2f}")
        print(f"   Final Balance: R$ {self.session.final_balance:.2f}")
        print(f"   Net P&L: R$ {self.session.net_pnl:+.2f} ({self.session.roi_pct:+.2f}%)")
        print(f"   Total Fees Paid: R$ {self.session.total_fees_paid:.2f}")
        
        print(f"\n🛡️ Anti-Loss Protection:")
        print(f"   Unprofitable sells prevented: ON")
        print(f"   Final positions protected: {len(self.active_positions)}")
        
        print(f"\n⏰ Completed: {self.session.end_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 80)


async def get_current_price_mock(symbol: str = "SOL-BRL") -> float:
    """Mock function to get current price - replace with real API call."""
    import random
    # Simulate SOL price around 180-220 BRL with volatility
    base_price = 200.0
    volatility = random.uniform(-0.05, 0.05)  # ±5% volatility
    return base_price * (1 + volatility)


async def run_live_trading_session():
    """Run 6-hour live trading session with real-time signal generation."""
    print("🚀 STARTING 6-HOUR LIVE TRADING SESSION")
    print("=" * 80)
    
    config = TradingConfig.from_env(".env")
    system = IntelligentTradingSystem(config)
    
    # Initialize session
    await system.initialize_session()
    
    # Calculate session duration
    session_duration_seconds = config.session_duration_hours * 3600  # 6h = 21600s
    signal_interval_seconds = 300  # Generate signal every 5 minutes (candle period)
    
    print(f"⏰ Session will run for {config.session_duration_hours} hours")
    print(f"📊 Generating signals every {signal_interval_seconds} seconds")
    print(f"🎯 Total expected signals: {session_duration_seconds // signal_interval_seconds}")
    print("=" * 80)
    
    start_time = datetime.now()
    signal_count = 0
    
    try:
        while True:
            current_time = datetime.now()
            elapsed_seconds = (current_time - start_time).total_seconds()
            
            # Check if session duration reached
            if elapsed_seconds >= session_duration_seconds:
                print(f"\n⏰ Session duration ({config.session_duration_hours}h) completed!")
                break
            
            # Get latest 5-minute candle
            try:
                latest_candle = await fetch_latest_candle(system.api_client)
                if not latest_candle:
                    await asyncio.sleep(signal_interval_seconds)
                    continue
                    
                signal_count += 1
                current_price = latest_candle['close']
                
                # Generate ML signal using trained model and latest candle
                signal = await generate_ml_signal(system.strategy, system.market_data, latest_candle, signal_count)
                
                if signal:
                    # Process the signal
                    success = await system.process_signal(signal)
                    
                    remaining_hours = (session_duration_seconds - elapsed_seconds) / 3600
                    print(f"📊 Signal #{signal_count} | "
                          f"Price: R$ {current_price:.2f} | "
                          f"Type: {signal['signal_type']} | "
                          f"Confidence: {signal['confidence']:.1%} | "
                          f"Remaining: {remaining_hours:.1f}h")
                    
                    if success:
                        print(f"✅ Signal executed successfully")
                    else:
                        print(f"⚠️ Signal not executed (protection/limits)")
                
                # Wait before next signal
                await asyncio.sleep(signal_interval_seconds)
                
            except Exception as e:
                print(f"❌ Error in trading loop: {e}")
                await asyncio.sleep(signal_interval_seconds)
                continue
    
    except KeyboardInterrupt:
        print(f"\n🛑 Session interrupted by user after {elapsed_seconds/3600:.1f}h")
    
    finally:
        # Finalize session
        print(f"\n📊 Finalizing session after {signal_count} signals...")
        await system.finalize_session()
        
        print("🏁 LIVE TRADING SESSION COMPLETED")
        print("=" * 80)


async def fetch_latest_candle(api_client, symbol: str = "SOL-BRL") -> Optional[Dict[str, Any]]:
    """Fetch the latest 5-minute candle from the real API."""
    try:
        if api_client:
            # Get the most recent candle using countback=1
            df = api_client.get_candles(
                symbol=symbol,
                timeframe="5m", 
                countback=1,
                use_cache=False  # Always get fresh data
            )
            
            if not df.empty:
                latest = df.iloc[-1]
                return {
                    'timestamp': latest.name,  # Use index as timestamp
                    'open': float(latest['open']),
                    'high': float(latest['high']),
                    'low': float(latest['low']),
                    'close': float(latest['close']),
                    'volume': float(latest['volume'])
                }
        
        # Fallback: simulate realistic candle if no API
        import random
        current_time = datetime.now()
        base_price = 200.0
        volatility = random.uniform(-0.02, 0.02)  # ±2% volatility
        close_price = base_price * (1 + volatility)
        
        # Create realistic OHLCV
        open_price = close_price * (1 + random.uniform(-0.005, 0.005))
        high_price = max(open_price, close_price) * (1 + random.uniform(0, 0.003))
        low_price = min(open_price, close_price) * (1 - random.uniform(0, 0.003))
        volume = random.uniform(800, 1500)
        
        return {
            'timestamp': current_time,
            'open': open_price,
            'high': high_price,
            'low': low_price,
            'close': close_price,
            'volume': volume
        }
        
    except Exception as e:
        print(f"❌ Error fetching candle: {e}")
        return None


async def generate_ml_signal(strategy, market_data: pd.DataFrame, latest_candle: Dict[str, Any], signal_count: int) -> Optional[Dict[str, Any]]:
    """Generate ML signal using trained model and latest candle."""
    try:
        # Convert latest candle to DataFrame format
        import pandas as pd
        
        candle_df = pd.DataFrame([latest_candle])
        
        # Append to historical data for feature calculation (SMAs need history)
        extended_data = pd.concat([market_data, candle_df], ignore_index=True)
        
        # Use only the latest row for prediction (but calculate features with full history)
        signal_result = strategy.generate_signal(extended_data.tail(1))
        
        if signal_result and signal_result.get('signal_type') != 'hold':
            return {
                'signal_type': signal_result['signal_type'],
                'price': latest_candle['close'],
                'confidence': signal_result.get('confidence', 0.5),
                'predictions': signal_result.get('predictions', {}),
                'reason': f"ML prediction on candle #{signal_count}",
                'timestamp': latest_candle['timestamp']
            }
        
        return None
        
    except Exception as e:
        print(f"❌ Error generating ML signal: {e}")
        return None


# Test function (kept for debugging)
async def test_intelligent_system():
    """Test the intelligent trading system with simulated signals."""
    print("Testing Intelligent Trading System...")
    
    config = TradingConfig.from_env()
    system = IntelligentTradingSystem(config)
    
    # Initialize
    await system.initialize_session()
    
    # Simulate some signals
    test_signals = [
        {
            'signal_type': 'buy',
            'price': 180.0,
            'confidence': 0.85,
            'predictions': {'tree': 0.4, 'mlp': 0.3, 'final': 0.37}
        },
        {
            'signal_type': 'sell', 
            'price': 175.0,  # Loss scenario
            'confidence': 0.80,
            'predictions': {'tree': -0.3, 'mlp': -0.4, 'final': -0.34}
        },
        {
            'signal_type': 'sell',
            'price': 190.0,  # Profit scenario
            'confidence': 0.90,
            'predictions': {'tree': -0.5, 'mlp': -0.2, 'final': -0.38}
        }
    ]
    
    for signal in test_signals:
        await system.process_signal(signal)
        await asyncio.sleep(1)  # Small delay
    
    # Finalize
    await system.finalize_session()
    
    print("Test completed!")

In [7]:
async def main(mode="live"):
    """
    Main function for Jupyter notebook execution
    Args:
        mode: "live" for live trading, "test" for simulation
    """
    if mode == "test":
        print("🧪 Running in TEST mode")
        return await test_intelligent_system()
    else:
        print("🔴 Running in LIVE TRADING mode")
        return await run_live_trading_session()

In [8]:
results = await main("test")

🧪 Running in TEST mode
Testing Intelligent Trading System...
Configuration loaded           budget=100.0 mode='test  # test or live' positions=5 telegram_enabled=True
SimpleMLEnsembleStrategy initialized asset=SOL-BRL historical_bars=2016 session_bars=360 strategy=SimpleMLEnsemble
Telegram service enabled       authorized_users_count=1 has_chat_id=True service=telegram
Initializing intelligent trading session system=intelligent_trading
Preparing ML strategy with fresh data system=intelligent_trading
Updating market data cache...  system=intelligent_trading
ℹ️  No API config provided, will create synthetic data
SOL-BRL Data Collection for Trading Exercise
🎭 Creating synthetic SOL-BRL data...
📅 Target period: 2025-08-01 12:00:00 → 2025-08-08 12:00:00
📊 Target bars: 2016 (5min intervals)
✅ Created 2016 synthetic bars
   Date range: 2025-08-01 12:00:00 → 2025-08-08 11:55:00
   Price range: 145.09 - 246.24 BRL
   Volume range: 1,027 - 17,339
💾 Data saved to: data/exercise/sol_data.db
💾 CSV 